In [1]:
!pip install boto3
!pip install python-dotenv
!pip install requests

   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ------ --------------------------------- 10.2/62.6 kB ? eta -:--:--
   -------------------------- ------------- 41.0/62.6 kB 667.8 kB/s eta 0:00:01
   ---------------------------------------- 62.6/62.6 kB 562.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/162.5 kB ? eta -:--:--
   -------------------- ------------------- 81.9/162.5 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 162.5/162.5 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ------------------------------------ --- 92.2/

In [5]:
import os
import boto3
import requests
from dotenv import load_dotenv

load_dotenv()
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.environ.get('REGION_NAME')

dynamodb = boto3.resource('dynamodb', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY, region_name=REGION_NAME)


In [20]:
table = dynamodb.Table('SPOTIFY_ZODIAC')

response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

print(f"there are {len(data)} entries")
print(f"\n{data}")

there are 1 entries

[{'count': Decimal('50'), 'Account_id': 'b571fd6b1cbd0247ac805858bdc52c13bbfc71f50082e0a7f1bd253bdf5a56c8', 'audio_features': [{'track_href': 'https://api.spotify.com/v1/tracks/1PJe0Vt9p0QaV4ZNHUwml0', 'loudness': '-10.769', 'liveness': '0.118', 'tempo': '110.088', 'valence': '0.497', 'instrumentalness': Decimal('0'), 'type': 'audio_features', 'danceability': '0.733', 'uri': 'spotify:track:1PJe0Vt9p0QaV4ZNHUwml0', 'speechiness': '0.0291', 'mode': Decimal('0'), 'duration_ms': Decimal('214922'), 'acousticness': '0.559', 'id': '1PJe0Vt9p0QaV4ZNHUwml0', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1PJe0Vt9p0QaV4ZNHUwml0', 'key': Decimal('0'), 'energy': '0.369', 'time_signature': Decimal('4')}, {'track_href': 'https://api.spotify.com/v1/tracks/45mvVEa99kOMcjHJpCykrw', 'loudness': '-9.505', 'liveness': '0.126', 'tempo': '169.941', 'valence': '0.708', 'instrumentalness': '5.51e-05', 'type': 'audio_features', 'danceability': '0.58', 'uri': 'spotify:track:45mv

In [23]:
access_token = 'BQD3il9bZBqGLaeF4kvnlMDYyWckgjjIXLQ1ESqAPyh-51nfQNNh3j-bO8gxN3LzcHgI54c5GDplTH_8dUwbxNmwqu2kmVf9JXclKOj2chX-bmrjuHQFEVa346KcNFGWbwZ-eL3DiJl-Ek0TsnodT3c_EsbtpZXf1HKND1JSQvBHfuL-HA2sZJiegZ2cOEhT'


tracks_table = dynamodb.Table('SPOTIFY_TRACKS')

headers = {
  'Authorization': f"Bearer {access_token}"
}

response = requests.get(url = 'https://api.spotify.com/v1/me/playlists?limit=50', headers = headers)

playlist_ids = []
if response.status_code == 200:
  raw_json = response.json()
  
  for playlist in raw_json['items']:
    playlist_ids.append(playlist['id'])
else:
  print(response.text)

num = 1
for playlist_id in playlist_ids:
  print(f"fetching playlist {playlist_id} ({num}/{len(playlist_ids)})")
  response = requests.get(url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?market=US&limit=50", headers = headers)

  if response.status_code == 200:
    raw_json = response.json()
    
    for item in raw_json['items']:
      track_id = item['track']['id']

      db_response = tracks_table.get_item(Key={
        'id': track_id,
      })

      track_ids = ''
      # add track if havent been added
      if ('Item' not in db_response):
        track_ids += f"{track_id},"

      track_ids = track_ids[:-1]

      query = {
        'ids': track_ids
      }

      if len(track_ids) == 0:
        break

      response = requests.get(url = 'https://api.spotify.com/v1/audio-features', headers = headers, params = query)

      if response.status_code == 200:
        raw_json = response.json()
        
        i = 0
        for audio_features in raw_json['audio_features']:
          for key, value in audio_features.items():
            if isinstance(value, float):
              raw_json['audio_features'][i][key] = str(value)
                  
          i += 1
        
        for track in raw_json['audio_features']:
          tracks_table.put_item(Item=track)

    num += 1

  else:
    print(response.text)

fetching playlist 7FaEz3qxsCRNySGytSmShN (1/28)
fetching playlist 1zqSZLDM3pjX8cnRdSugZi (2/28)
fetching playlist 1BQhZk5Ix5aK6T4PXIwm6O (3/28)
fetching playlist 0MkomoU7mzw57gKkpkzpsL (4/28)
fetching playlist 3it63qauS2sCnWYJEnw2GV (5/28)
fetching playlist 2y1bFBnYl61ZbfUy5HdJRt (6/28)
fetching playlist 4UcNdsCKwuj84A6HNXnIrH (7/28)
fetching playlist 7ABuEPkVnHY0O8MhXC487c (8/28)
fetching playlist 1NoM1r2yu3sqMSgAZKUNvw (9/28)
fetching playlist 03pnSuMa7pu1wMJ3f2RPig (10/28)
fetching playlist 6Hv67Fe0Bco6ARdZwqfofH (11/28)
fetching playlist 1HvgaonFwkONX1ZFJh6mMk (12/28)
fetching playlist 6ExKMWX06VQHM6FUUbHpAy (13/28)
fetching playlist 2w3nd2BiXpA15ZlcaMCyr2 (14/28)
fetching playlist 5vzj0g9AEgpuYPh9a1bWmk (15/28)
fetching playlist 6p0inNUHd6gGgrvlFMtusO (16/28)
fetching playlist 0kaVXOyRVVHA7e2kJZBjZY (17/28)
fetching playlist 74nvuaZEC1S28d9wIcLfD4 (18/28)
fetching playlist 35IKH1yUtRFP9ED952p2CE (19/28)
fetching playlist 12XfMyLzXOhH29OKunqBPV (20/28)
fetching playlist 2Hft9JBl9sP